# From Data to APIs

In [ ]:
from dartfx.usblstoolkit import model
from dartfx.datapublisher import postgres
from dartfx.apipublisher import mtnards
import os
import logging
logging.basicConfig(format='%(asctime)s %(levelname)s: %(message)s', level="DEBUG")

In [ ]:
# Setup BLS database
repo = model.BlsRepository('/Volumes/Seagate/DataArchive/HighValueDataNetwork/us-bls/repository')
db_id = 'cw'
db = model.BlsDatabase(repo, db_id)

# setup postgres
pg_server = postgres.PostgresServer(host="pg17215.highvaluedata.net", password="postgres", schema="us_bls")
#pg_server = postgres.PostgresServer(host="localhost", password="postgres", schema="us_bls"

In [ ]:
# Publish data to Postgres
pg_publisher_config = postgres.PostgresDdiCodebookPublisherConfig()
pg_publisher_config.if_exists = 'fail'
pg_publisher_config.if_exists = 'replace'
pg_publisher = postgres.PostgresDdiCodebookPublisher(pg_server, pg_publisher_config)
for id,datafile in db.datafiles.items():
    if datafile.number != 1:
        #continue
        pass
    pg_publisher_config.ddi_filename = datafile.ddic_filepath
    pg_publisher.publish()

In [ ]:
# Publish to MTNA RDS
pg_vpc_server = postgres.PostgresServer(host="pg.hvdn172", password=pg_server.password, schema=pg_server.schema)
rds_config = mtnards.MtnaRdsPublisherConfig()

# Where to publish in RDS 
rds_config.rds_host = "https://www172.highvaluedata.net"
rds_config.rds_api_key = "S5pBrUZ2nxiVRJPs" # default woud pick from environment variable
rds_config.rds_catalog_id = f"us_bls_{db.id}"
rds_config.rds_make_public = True
rds_config.rds_overwrite = False
rds_config.rds_ssl_verify = False

# Where is the data coming from 
rds_config.sql_connection_string = pg_vpc_server.jdbc_connection_string
rds_config.sql_username = pg_vpc_server.username
rds_config.sql_password = pg_vpc_server.password

# Postman
#rds_config.postman_workspace_id = '15c355c1-942c-4268-818e-fe37b8e591ab'
#rds_config.postman_api_key = os.getenv('POSTMAN_API_KEY')


datafile = db.datafile0
rds_config.rds_data_product_id = datafile.topic 
rds_config.sql_table = datafile.filename.rsplit(".", 1)[0].replace(".","_")
rds_config.postman_collection_filename = os.path.join(db.products_dir, datafile.filename+'.postman.json')
rds_config.ddi_filepath = datafile.ddic_filepath

rds_publisher = mtnards.MtnaRdsPublisher(rds_config)

In [5]:
rds_publisher.publish()

2024-04-29 15:39:18,751 INFO: === Pre-publishing checks ===
2024-04-29 15:39:18,806 INFO: Loading DDI file
2024-04-29 15:39:18,871 INFO: DDI file contains single file fileDscr
2024-04-29 15:39:18,872 INFO: DDI file looks good
2024-04-29 15:39:18,922 DEBUG: Starting new HTTPS connection (1): www172.highvaluedata.net:443
/Users/pascal/anaconda3/lib/python3.10/site-packages/urllib3/connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www172.highvaluedata.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
2024-04-29 15:39:19,031 DEBUG: https://www172.highvaluedata.net:443 "GET /rds/api/server/info HTTP/1.1" 200 None
2024-04-29 15:39:19,041 DEBUG: Starting new HTTPS connection (1): www172.highvaluedata.net:443
/Users/pascal/anaconda3/lib/python3.10/site-packages/urllib3/connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is b